In [1]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
import os
import sys
sys.path.append(os.path.abspath('../'))
from tqdm import tqdm
from pathlib import Path
from collections import Counter
import multiprocessing as mp

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa import stattools
from scipy.stats import entropy, skew, kurtosis

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from skin_lesion_cad.features.colour import ColorFeaturesExtractor

from sklearn.svm import SVC

from sklearn.base import TransformerMixin, BaseEstimator


In [3]:
def process_img(x):
    img_path, cfe = x
    img = cv2.imread(str(img_path))
    img = cv2.medianBlur(img, 3)
    
    mask = cv2.imread(str(img_path).replace('inpaint', 'mask'), cv2.IMREAD_GRAYSCALE)

    img_name = img_path.name.split('_')[0]
    imf_fearures = cfe.extract_masked(img, mask)
    imf_fearures['name'] = img_name
    return imf_fearures

In [4]:
class ColorTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.scaler = StandardScaler()
    def fit(self, X, y=None):
        X[np.isnan(X)] = 0
        X[np.isinf(X)] = 0
        return self.scaler.fit(X)
    def transform(self, X, y=None):
        X[np.isnan(X)] = 0
        X[np.isinf(X)] = 0
        return self.scaler.transform(X)

In [5]:
proc_data = Path().resolve().parent / 'data/processed'

color_spaces = {'bgr':cv2.COLOR_RGB2BGR, 'hsv':cv2.COLOR_RGB2HSV, 'YCrCb':cv2.COLOR_RGB2YCrCb}

In [6]:
chl = 'chall1'
mode = 'train'

test_path = proc_data / f'{chl}/{mode}'

np.random.seed(42)
all_imgs = np.random.choice(list(test_path.glob('**/*inpaint_0_5*')), size=5000)
Counter([1 if 'nevus' in str(x) else 0 for x in all_imgs])

Counter({1: 2569, 0: 2431})

In [7]:
gird = [{'sp': 10, 'sr': 15}, #initial one
        {'sp': 5, 'sr': 15},
        {'sp': 15, 'sr': 15},
        {'sp': 20, 'sr': 15},
        {'sp': 30, 'sr': 15},
        {'sp': 10, 'sr': 10},
        {'sp': 10, 'sr': 5},
        {'sp': 10, 'sr': 20},
        {'sp': 20, 'sr': 30},
        {'sp': 5, 'sr': 8},
        ] # spatial and color range radius

In [8]:
grid_search_results = []

for mshift_params in tqdm(gird): 

    cfe = ColorFeaturesExtractor(color_spaces, mshift_params)
    dfs = []

    candidates_features = []
    with mp.Pool(8) as pool:
        for result in pool.imap(process_img, zip(all_imgs, [cfe]*len(all_imgs))):
            candidates_features.append(result)
            
    dfs.append(pd.DataFrame(candidates_features))

    dfs = pd.concat(dfs)
    dfs['class'] = [0 if 'nev' in x else 1 for x in dfs['name']]


    pipe = Pipeline([('scaler', ColorTransformer()), ('svc', SVC())])

    X = dfs.drop(columns=['name', 'class']).values
    y = dfs['class'].values

    cv_results = cross_validate(pipe, X, y, cv=5, scoring=['accuracy', 'roc_auc'], n_jobs=8, return_train_score=True)

    res = {'mshift_params':mshift_params,
        'train_acc':cv_results['train_accuracy'].mean(),
        'test_acc':cv_results['test_accuracy'].mean(),
        'train_roc_auc':cv_results['train_roc_auc'].mean(),
        'test_roc_auc':cv_results['test_roc_auc'].mean()}

    grid_search_results.append(res)

 80%|████████  | 8/10 [1:15:41<17:38, 529.03s/it]/media/vzalevskyi/DATA/ProjectsData/skin-lesion-cad/skin_lesion_cad/features/colour.py:38: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res[f'{clrsp}_{clrsp[clrsp_idx]}_skew'] = skew(pixels)
/media/vzalevskyi/DATA/ProjectsData/skin-lesion-cad/skin_lesion_cad/features/colour.py:39: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res[f'{clrsp}_{clrsp[clrsp_idx]}_kurt'] = kurtosis(pixels)
/media/vzalevskyi/DATA/ProjectsData/skin-lesion-cad/skin_lesion_cad/features/colour.py:38: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res[f'{clrsp}_{clrsp[clrsp_idx]}_skew'] = ske

In [10]:
mshift_res = pd.DataFrame(grid_search_results)
mshift_res.sort_values(by='test_acc', ascending=False)

,mshift_params,train_acc,test_acc,train_roc_auc,test_roc_auc
9,"{'sp': 5, 'sr': 8}",0.81200,0.7758,0.892049,0.854345
5,"{'sp': 10, 'sr': 10}",0.81125,0.7734,0.890606,0.852845
6,"{'sp': 10, 'sr': 5}",0.81245,0.7728,0.893073,0.854433
3,"{'sp': 20, 'sr': 15}",0.80585,0.7716,0.888181,0.850934
1,"{'sp': 5, 'sr': 15}",0.80985,0.7708,0.890480,0.851278
2,"{'sp': 15, 'sr': 15}",0.80760,0.7704,0.889833,0.851539
8,"{'sp': 20, 'sr': 30}",0.80725,0.7700,0.885991,0.848625
0,"{'sp': 10, 'sr': 15}",0.80820,0.7694,0.890631,0.851052
7,"{'sp': 10, 'sr': 20}",0.80950,0.7680,0.890384,0.850978
4,"{'sp': 30, 'sr': 15}",0.80840,0.7666,0.888914,0.848440
